## Using SE-ResNeXt for image classification
- Writes test classification to csv using test generator
- Squeeze and Excitation networks - https://arxiv.org/abs/1709.01507
- ResNeXt architecture - https://arxiv.org/abs/1611.05431
- Keras model - https://github.com/osmr/imgclsmob
- Kerascv - https://pypi.org/project/kerascv/

In [3]:
import sys
import numpy
import matplotlib
import pandas
import keras

print('Python: {}'.format(sys.version))
print('Numpy: {}'.format(numpy.__version__))
print('Matplotlib: {}'.format(matplotlib.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Keras: {}'.format(keras.__version__))

Using TensorFlow backend.


Python: 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Numpy: 1.16.4
Matplotlib: 3.0.2
Pandas: 0.23.4
Keras: 2.2.4


In [4]:
from kerascv.model_provider import get_model as kecv_get_model

#set pretrained = True for Imagenet weights
base_model = kecv_get_model(
    name = "seresnext101_32x4d",
    pretrained = False)

Also available are SE-ResNeXts with different cardinalities and layers:
- 'seresnext50_32x4d': seresnext50_32x4d,
- 'seresnext101_32x4d': seresnext101_32x4d,
- 'seresnext101_64x4d': seresnext101_64x4d,

In [5]:
#set trainable = false for transfer learning 

for layer in base_model.layers[:]:
    layer.trainable = True

Replace output dense layer of 1000 classes to your classification needs 

In [6]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
features/init_block/conv/conv ( (None, 112, 112, 64) 9408        lambda_1[0][0]                   
__________________________________________________________________________________________________
features/init_block/conv/bn (Ba (None, 112, 112, 64) 256         features/init_block/conv/conv[0][
__________________________________________________________________________________________________
features/i

__________________________________________________________________________________________________
lambda_261 (Lambda)             (None, 30, 30, 16)   0           lambda_234[0][0]                 
__________________________________________________________________________________________________
lambda_262 (Lambda)             (None, 30, 30, 16)   0           lambda_234[0][0]                 
__________________________________________________________________________________________________
lambda_263 (Lambda)             (None, 30, 30, 16)   0           lambda_234[0][0]                 
__________________________________________________________________________________________________
lambda_264 (Lambda)             (None, 30, 30, 16)   0           lambda_234[0][0]                 
__________________________________________________________________________________________________
lambda_265 (Lambda)             (None, 30, 30, 16)   0           lambda_234[0][0]                 
__________

__________________________________________________________________________________________________
features/stage3/unit8/se/sigmoi (None, 1, 1, 1024)   0           features/stage3/unit8/se/conv2[0]
__________________________________________________________________________________________________
features/stage3/unit8/se/mul (M (None, 14, 14, 1024) 0           features/stage3/unit8/body/conv3/
                                                                 features/stage3/unit8/se/sigmoid[
__________________________________________________________________________________________________
features/stage3/unit8/add (Add) (None, 14, 14, 1024) 0           features/stage3/unit8/se/mul[0][0
                                                                 features/stage3/unit7/activ[0][0]
__________________________________________________________________________________________________
features/stage3/unit8/activ (Ac (None, 14, 14, 1024) 0           features/stage3/unit8/add[0][0]  
__________

__________________________________________________________________________________________________
features/stage3/unit16/body/con (None, 14, 14, 16)   2304        lambda_759[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit16/body/con (None, 14, 14, 16)   2304        lambda_760[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit16/body/con (None, 14, 14, 16)   2304        lambda_761[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit16/body/con (None, 14, 14, 512)  0           features/stage3/unit16/body/conv2
                                                                 features/stage3/unit16/body/conv2
                                                                 features/stage3/unit16/body/conv2
          

features/stage4/unit2/body/conv (None, 7, 7, 32)     9216        lambda_1029[0][0]                
__________________________________________________________________________________________________
features/stage4/unit2/body/conv (None, 7, 7, 32)     9216        lambda_1030[0][0]                
__________________________________________________________________________________________________
features/stage4/unit2/body/conv (None, 7, 7, 32)     9216        lambda_1031[0][0]                
__________________________________________________________________________________________________
features/stage4/unit2/body/conv (None, 7, 7, 32)     9216        lambda_1032[0][0]                
__________________________________________________________________________________________________
features/stage4/unit2/body/conv (None, 7, 7, 32)     9216        lambda_1033[0][0]                
__________________________________________________________________________________________________
features/s

In [7]:
base_model.layers[-2]

Create a new model and remove the top dense layer

In [8]:
from keras.models import Model
from keras.layers.core import Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D
from keras.layers import Input
from keras.layers.merge import concatenate, add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras_applications.imagenet_utils import _obtain_input_shape
import keras.backend as K

model = Model(
    inputs = base_model.input,
    outputs = base_model.get_layer(index = -2).output)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
features/init_block/conv/conv ( (None, 112, 112, 64) 9408        lambda_1[0][0]                   
__________________________________________________________________________________________________
features/init_block/conv/bn (Ba (None, 112, 112, 64) 256         features/init_block/conv/conv[0][
__________________________________________________________________________________________________
features/i

features/stage3/unit3/body/conv (None, 14, 14, 512)  0           features/stage3/unit3/body/conv2/
__________________________________________________________________________________________________
features/stage3/unit3/body/conv (None, 14, 14, 1024) 524288      features/stage3/unit3/body/conv2/
__________________________________________________________________________________________________
features/stage3/unit3/body/conv (None, 14, 14, 1024) 4096        features/stage3/unit3/body/conv3/
__________________________________________________________________________________________________
features/stage3/unit3/se/pool ( (None, 1, 1, 1024)   0           features/stage3/unit3/body/conv3/
__________________________________________________________________________________________________
features/stage3/unit3/se/conv1  (None, 1, 1, 64)     65600       features/stage3/unit3/se/pool[0][
__________________________________________________________________________________________________
features/s

__________________________________________________________________________________________________
lambda_673 (Lambda)             (None, 16, 16, 16)   0           lambda_663[0][0]                 
__________________________________________________________________________________________________
lambda_674 (Lambda)             (None, 16, 16, 16)   0           lambda_663[0][0]                 
__________________________________________________________________________________________________
lambda_675 (Lambda)             (None, 16, 16, 16)   0           lambda_663[0][0]                 
__________________________________________________________________________________________________
lambda_676 (Lambda)             (None, 16, 16, 16)   0           lambda_663[0][0]                 
__________________________________________________________________________________________________
lambda_677 (Lambda)             (None, 16, 16, 16)   0           lambda_663[0][0]                 
__________

lambda_1003 (Lambda)            (None, 16, 16, 32)   0           lambda_993[0][0]                 
__________________________________________________________________________________________________
lambda_1004 (Lambda)            (None, 16, 16, 32)   0           lambda_993[0][0]                 
__________________________________________________________________________________________________
lambda_1005 (Lambda)            (None, 16, 16, 32)   0           lambda_993[0][0]                 
__________________________________________________________________________________________________
lambda_1006 (Lambda)            (None, 16, 16, 32)   0           lambda_993[0][0]                 
__________________________________________________________________________________________________
lambda_1007 (Lambda)            (None, 16, 16, 32)   0           lambda_993[0][0]                 
__________________________________________________________________________________________________
lambda_100

Replace the dense layer with a dense layer with 5 units for 5 classes (for example)

In [9]:
from keras.layers import Activation, Dense, Dropout

x = model.output
output = Dense(5)(x)
output = Activation('softmax')(output)

model = Model(
    inputs = base_model.input,
    outputs = output)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
features/init_block/conv/conv ( (None, 112, 112, 64) 9408        lambda_1[0][0]                   
__________________________________________________________________________________________________
features/init_block/conv/bn (Ba (None, 112, 112, 64) 256         features/init_block/conv/conv[0][
__________________________________________________________________________________________________
features/i

features/stage3/unit3/se/mul (M (None, 14, 14, 1024) 0           features/stage3/unit3/body/conv3/
                                                                 features/stage3/unit3/se/sigmoid[
__________________________________________________________________________________________________
features/stage3/unit3/add (Add) (None, 14, 14, 1024) 0           features/stage3/unit3/se/mul[0][0
                                                                 features/stage3/unit2/activ[0][0]
__________________________________________________________________________________________________
features/stage3/unit3/activ (Ac (None, 14, 14, 1024) 0           features/stage3/unit3/add[0][0]  
__________________________________________________________________________________________________
features/stage3/unit4/body/conv (None, 14, 14, 512)  524288      features/stage3/unit3/activ[0][0]
__________________________________________________________________________________________________
features/s

features/stage3/unit13/body/con (None, 14, 14, 16)   2304        lambda_647[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit13/body/con (None, 14, 14, 16)   2304        lambda_648[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit13/body/con (None, 14, 14, 16)   2304        lambda_649[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit13/body/con (None, 14, 14, 16)   2304        lambda_650[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit13/body/con (None, 14, 14, 16)   2304        lambda_651[0][0]                 
__________________________________________________________________________________________________
features/s

features/stage3/unit23/body/con (None, 14, 14, 16)   2304        lambda_981[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit23/body/con (None, 14, 14, 16)   2304        lambda_982[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit23/body/con (None, 14, 14, 16)   2304        lambda_983[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit23/body/con (None, 14, 14, 16)   2304        lambda_984[0][0]                 
__________________________________________________________________________________________________
features/stage3/unit23/body/con (None, 14, 14, 16)   2304        lambda_985[0][0]                 
__________________________________________________________________________________________________
features/s

ImageDataGenerator for training and testing

In [ ]:
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    fill_mode = 'reflect',
    rotation_range = 50,
    horizontal_flip = True,
    vertical_flip = True,
    validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        subset = "training")

valid_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        subset = "validation")

In [ ]:
x_batch, y_batch = next(train_generator)

Plot the ImageDataGenerator results

In [ ]:
import matplotlib.pyplot as plt

for i in range (0,20):
    image = x_batch[i]
    plt.imshow(image)
    plt.show()

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2,
                              patience=4, 
                              min_lr= 1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr = 2e-4),
              metrics = ['accuracy'])

In [ ]:
model.fit_generator(generator = train_generator,
                   validation_steps = valid_generator.n // valid_generator.batch_size + 1,
                   validation_data = valid_generator,
                   steps_per_epoch = train_generator.n // train_generator.batch_size + 1,
                    epochs = 10,
                   callbacks = [reduce_lr])

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255)

test_generator=test_datagen.flow_from_directory(
                directory="./test",
                batch_size=20,
                seed=42,
                shuffle=False,
                class_mode=None,
                target_size=(224,224))

STEP_SIZE_TEST = test_generator.n // test_generator.batch_size + 1
test_generator.reset()


pred=model.predict_generator(test_generator,
                steps=STEP_SIZE_TEST,
                verbose=1)

Output predictions to a csv

In [ ]:
import numpy as np
import pandas as pd 

predicted_class_indices = np.argmax(pred, axis=1)   

In [ ]:
predicted_class_indices[:10]

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames

results__ = []
for result in predicted_class_indices :
    if result == 0:
        results__.append('class1')
    elif result == 1:
        results__.append('class2')
    elif result == 2:
        results__.append('class3')
    elif result == 3: 
        results__.append('class4')
    elif result == 4: 
        results__.append('class5')

print(results__)

results = pd.DataFrame({"Category":results__,
                      "Id":filenames})

In [ ]:
results.head()

In [ ]:
results.Category.value_counts()

In [ ]:
results.head()

In [ ]:
results.to_csv(
    "submission.csv",
    index=False)